In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../Titanic"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

# 数据预览

In [ ]:
data_train = pd.read_csv("../Titanic/train.csv")
data_test = pd.read_csv("../Titanic/test.csv")

# 分别输出训练数据和验证数据的基本信息。对数据的规模、各特征的数据类型及是否缺失有一个总体了解
data_train.info()  # 数据统计信息

data_test.info()

# data_train.describe() # 查看数值型列的汇总统计
# 总生存率

# 数据分析

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Pclass         891 non-null int64
fig = plt.figure()
Pclass_survived_rate = (data_train.groupby(['Pclass']).sum() / data_train.groupby(['Pclass']).count())['Survived']
Pclass_survived_rate.plot(kind='bar')
plt.title('Sex_survived_rate') 
plt.show()
# 乘客社会等级越高，幸存率越高

In [ ]:
# Name           891 non-null object

In [ ]:
# Sex            891 non-null object
fig = plt.figure()
Sex_survived_rate = (data_train.groupby(['Sex']).sum() / data_train.groupby(['Sex']).count())['Survived']
Sex_survived_rate.plot(kind='bar')
plt.title('Sex_survived_rate') 
plt.show()
# 女性幸存率远高于男性

In [ ]:
# Age            714 non-null float64

# 对于连续型数值变量，可以先做特征离散化
age_clean_date = data_train[~np.isnan(data_train['Age'])] #去除年龄数据中的NaN 
ages = np.arange(0,81,5) #0~80岁，每5岁一段（年龄最大80岁） 
age_cut = pd.cut(age_clean_date.Age, ages) 
age_cut_grouped = age_clean_date.groupby(age_cut) 
age_Survival_Rate = (age_cut_grouped.sum() / age_cut_grouped.count())['Survived'] #计算每年龄段的幸存率 
age_Survival_Rate.plot(kind='bar')
plt.title('Age_group_survived_rate') 
plt.show()
# 未成年人幸存率高于成年人

In [ ]:
# SibSp          891 non-null int64
fig = plt.figure()
SibSp_survived_rate = (data_train.groupby(['SibSp']).sum() / data_train.groupby(['SibSp']).count())['Survived']
SibSp_survived_rate.plot(kind='bar')
plt.title('SibSp_survived_rate') 
plt.show()
# 配偶及兄弟姐妹数适中的乘客更易幸存

In [ ]:
# Parch          891 non-null int64
fig = plt.figure()
Parch_survived_rate = (data_train.groupby(['Parch']).sum() / data_train.groupby(['Parch']).count())['Survived']
Parch_survived_rate.plot(kind='bar')
plt.title('Parch_survived_rate') 
plt.show()
# FamilySize为2到4的乘客幸存可能性较高

In [ ]:
# Ticket         891 non-null object

In [ ]:
# Fare           891 non-null float64
fig = plt.figure()
Fare_survived_rate = (data_train.groupby(['Fare']).sum() / data_train.groupby(['Fare']).count())['Survived']
Fare_survived_rate.plot()
plt.title('Fare_survived_rate') 
plt.show()
# 支出船票费越高幸存率越高

In [ ]:
# Cabin          204 non-null object
fig = plt.figure()
Cabin_survived_rate = (data_train.groupby(['Cabin']).sum() / data_train.groupby(['Cabin']).count())['Survived']
Cabin_survived_rate.plot()
plt.title('Cabin_survived_rate') 
plt.show()
# 不同仓位的乘客幸存率不同

In [ ]:
# Embarked       889 non-null object
fig = plt.figure()
Embarked_survived_rate = (data_train.groupby(['Embarked']).sum() / data_train.groupby(['Embarked']).count())['Survived']
Embarked_survived_rate.plot(kind='bar')
plt.title('Embarked_survived_rate') 
plt.show()
# Embarked为S的乘客幸存率较低

# 特征选择

In [ ]:
selected_features = ['Pclass', 'Sex', 'Age', 'Embarked', 'SibSp', 'Parch', 'Fare']

X_train = data_train[selected_features]
X_test = data_test[selected_features]

y_train = data_train['Survived']

# 填补缺失值

In [ ]:
# Embarked
print(X_train['Embarked'].value_counts())
print(X_test['Embarked'].value_counts())

# 对于这种类别型的特征，使用出现频率最高的特征值来填充
X_train['Embarked'].fillna('S', inplace=True) # 替换空值，inplace=True直接修改
X_test['Embarked'].fillna('S', inplace=True)

In [ ]:
# Age

# 对于这种数值型的特征，用求平均值或中位数来填充
X_train['Age'].fillna(X_train['Age'].mean(), inplace=True)  # mean均值，median中位数
X_test['Age'].fillna(X_test['Age'].mean(), inplace=True)

# Fare
X_test['Fare'].fillna(X_test['Fare'].mean(), inplace=True)

In [ ]:
# 重新查验数据
print(X_train.info())
print(X_test.info())

In [ ]:
# 采用DictVectorizer对特征向量化
from sklearn.feature_extraction import DictVectorizer

dict_vec = DictVectorizer(sparse=False)
X_train = dict_vec.fit_transform(X_train.to_dict(orient="record"))
print(dict_vec.feature_names_)

X_test = dict_vec.fit_transform(X_test.to_dict(orient='record'))
print(X_test.feature_names_)

In [ ]:
# 随机森林
from sklearn.ensemble import RandomForestClassifier

# 使用默认配置初始化
rfc = RandomForestClassifier()

